# #16 atmaCup

---

## セットアップ

### ライブラリのインストール

In [ ]:
# frequency-based regularization版iALSを共役勾配法で解く高速なライブラリ
# - 公式リポジトリ
#   - https://github.com/tohtsky/irspack
# - 開発者による解説記事
#   - https://engineering.visional.inc/blog/393/ials-revisited/
!pip install irspack

### ライブラリの読み込み

In [ ]:
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from irspack import (
    df_to_sparse,
    IALSRecommender,
    Evaluator,
    ItemIDMapper
)
from tqdm.auto import tqdm

In [ ]:
union = lambda x, y: x + y - x.multiply(y)

### データの読み込み

In [ ]:
trainvalid_log = pd.read_csv('train_log.csv')
trainvalid_label = pd.read_csv('train_label.csv')
test_log = pd.read_csv('test_log.csv')
test_session = pd.read_csv('test_session.csv')
yado = pd.read_csv('yado.csv')
sample_submission = pd.read_csv('sample_submission.csv')

## 前処理

### セッションと宿の一覧を作成

In [ ]:
# セッションの一覧
trainvalid_user_ids = trainvalid_log['session_id'].drop_duplicates().to_list()
test_user_ids = test_log['session_id'].drop_duplicates().to_list()

# 宿の一覧
item_ids = yado['yad_no'].drop_duplicates().to_list()

### 学習データと検証データに分割

精度検証のため `train_log.csv` を学習データと検証データに分割します。セッションが豊富に存在するため、今回はクロスバリデーションではなくホールドアウトにしました。

In [ ]:
train_user_ids, valid_user_ids = train_test_split(trainvalid_user_ids, test_size=0.2, random_state=0)
train_log = trainvalid_log.loc[lambda df: df['session_id'].isin(train_user_ids), :].copy()
valid_log = trainvalid_log.loc[lambda df: df['session_id'].isin(valid_user_ids), :].copy()
train_label = trainvalid_label.loc[lambda df: df['session_id'].isin(train_user_ids), :].copy()
valid_label = trainvalid_label.loc[lambda df: df['session_id'].isin(valid_user_ids), :].copy()

### ログを行列へ変換

- 行：セッション (※ `*_user_ids` 順にソートされている)
- 列：宿 (※ `item_ids` 順にソートされている)
- 成分：出現 or 予約なら 1、それ以外なら 0

In [ ]:
def log_to_matrix(log, label, user_ids, item_ids):
    # 出現した宿
    matrix_x, _, _ = df_to_sparse(
        df = log[['session_id', 'yad_no']].drop_duplicates(),
        user_column='session_id',
        item_column='yad_no',
        user_ids=user_ids,
        item_ids=item_ids
    )

    # 予約された宿
    if label is not None:
        matrix_y, _, _ = df_to_sparse(
            df=label,
            user_column='session_id',
            item_column='yad_no',
            user_ids=user_ids,
            item_ids=item_ids
        )
    else:
        matrix_y = None

    # レコメンド対象外の宿 (各セッション内で最後に出現する宿)
    matrix_mask, _, _ = df_to_sparse(
        df = (
            log
            .merge(
                log.groupby('session_id')['seq_no'].max().rename('seq_no_max'),
                how='left',
                on='session_id'
            )
            .loc[lambda df: df['seq_no'] == df['seq_no_max'], :]
        ),
        user_column='session_id',
        item_column='yad_no',
        user_ids=user_ids,
        item_ids=item_ids
    )

    return matrix_x, matrix_y, matrix_mask

In [ ]:
train_matrix_x, train_matrix_y, train_matrix_mask = \
    log_to_matrix(train_log, train_label, train_user_ids, item_ids)
valid_matrix_x, valid_matrix_y, valid_matrix_mask = \
    log_to_matrix(valid_log, valid_label, valid_user_ids, item_ids)
test_matrix_x, test_matrix_y, test_matrix_mask = \
    log_to_matrix(test_log, None, test_user_ids, item_ids)

### ログが2件以上あるセッションのインデックスを抽出

ログが1件しかないセッションはノイズになる可能性があるため、学習時に取り除けるよう、該当セッションを取り除いたインデックスをあらかじめ作っておきます。

In [ ]:
def get_user_ids_seq_no_max_over_1_index(log, user_ids):
    user_ids_seq_no_max_over_1 = (
        log
        .groupby('session_id')['seq_no'].max()
        .loc[lambda s: s > 0]
        .index
        .to_list()
    )
    user_ids_seq_no_max_over_1_set = set(user_ids_seq_no_max_over_1)
    user_ids_seq_no_max_over_1_index = [
        i
        for i, user_id in enumerate(user_ids)
        if user_id in user_ids_seq_no_max_over_1_set
    ]

    return user_ids_seq_no_max_over_1_index

In [ ]:
# train_user_ids_seq_no_max_over_1_index = \
#     get_user_ids_seq_no_max_over_1_index(train_log, train_user_ids)
# valid_user_ids_seq_no_max_over_1_index = \
#     get_user_ids_seq_no_max_over_1_index(valid_log, valid_user_ids)
test_user_ids_seq_no_max_over_1_index = \
    get_user_ids_seq_no_max_over_1_index(test_log, test_user_ids)

## 学習

### ハイパラ調整と精度検証

In [ ]:
valid_evaluator = Evaluator(
    valid_matrix_y,
    target_metric='map',
    cutoff=10,
    masked_interactions=valid_matrix_mask
)

In [ ]:
# # Optunaでハイパラ調整
# best_params, validation_results_df = IALSRecommender.tune_doubling_dimension(
#     scipy.sparse.vstack([
#         valid_matrix_x,
#         union(train_matrix_x, train_matrix_y)
#     ]),
#     valid_evaluator,
#     initial_dimension=200,
#     maximal_dimension=1600,
#     storage='sqlite:///optuna.db',
#     random_seed=0
# )

# ハイパラ調整の結果 (メモリ不足で中断したので最適じゃないかも……)
best_params = {
    'n_components': 1600,
    'alpha0': 0.001297421599991797,
    'reg': 0.010800184775061342,
    'train_epochs': 3
}

In [ ]:
valid_recommender = IALSRecommender(
    scipy.sparse.vstack([
        valid_matrix_x,
        union(train_matrix_x, train_matrix_y)
    ]),
    **best_params,
    random_seed=0
).learn()
valid_evaluator.get_score(valid_recommender)

### 全データ学習

In [ ]:
test_recommender = IALSRecommender(
    scipy.sparse.vstack([
        test_matrix_x,
        union(train_matrix_x, train_matrix_y),
        union(valid_matrix_x, valid_matrix_y),
        # 学習データ-テストデータ間のシフト対策として、テストデータを10倍にする。
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :],
        test_matrix_x[test_user_ids_seq_no_max_over_1_index, :]
    ]),
    **best_params,
    random_seed=0
).learn()

## 予測

In [ ]:
def get_candidates(recommender, offset, matrix_mask, user_ids, item_ids, num_candidates):
    batch_size = 10000
    id_mapper = ItemIDMapper(item_ids)
    cantidates = []
    for begin in tqdm(range(0, len(user_ids), batch_size)):
        end = min(begin + batch_size, len(user_ids))
        score = recommender.get_score_block(begin + offset, end + offset)
        score[matrix_mask[begin:end, :].nonzero()] = -np.inf
        cantidates += id_mapper.score_to_recommended_items_batch(score, cutoff=num_candidates)
    cantidates = (
        pd.DataFrame(
            [
                (user_ids[user_ids_index], rank + 1, yad_no, score)
                for user_ids_index, cantidates_per_user in enumerate(cantidates)
                for rank, (yad_no, score) in enumerate(cantidates_per_user)
            ],
            columns = ['session_id', 'rank', 'yad_no', 'score']
        )
    )

    return cantidates

In [ ]:
valid_cantidates = get_candidates(
    valid_recommender,
    0,
    valid_matrix_mask,
    valid_user_ids,
    item_ids,
    10
)

In [ ]:
test_cantidates = get_candidates(
    test_recommender,
    0,
    test_matrix_mask,
    test_user_ids,
    item_ids,
    10
)

## 後処理

[「train_logにおいてsessionの最後のseq_noと途中のseq_noの差分を取ったときに奇数番目の宿が選ばれやすい」](https://www.guruguru.science/competitions/22/discussions/b7abc605-9025-4a64-911e-2c760523db09/) で指摘されている通り、本コンペでは宿の出現順序が大きな意味を持っています。一方で、iALSは宿の出現順序を加味できません。そこで、iALSの予測結果に対して宿の出現順序に関するルールを後処理で適用します。

In [ ]:
# 各セッションのログの最後から2番目の宿をランキング1位に持ってくる。
def add_seq_no_reverse_1_yad(cantidates, log):
    # 各セッションのログの最後から2番目の宿
    seq_no_reverse_1_yad = (
        log
        .merge(
            log.groupby('session_id')['seq_no'].max().rename('seq_no_max'),
            how='left',
            on='session_id'
        )
        .merge(
            log.groupby('session_id')['yad_no'].nunique().rename('yad_no_nunique'),
            how='left',
            on='session_id'
        )
        .assign(seq_no_reverse = lambda df: df['seq_no_max'] - df['seq_no'])
        .loc[lambda df: (df['seq_no_reverse'] == 1) & (df['yad_no_nunique'] == 2), :]
        .assign(rank = -1)
        .loc[:, ['session_id', 'rank', 'yad_no']]
    )

    cantidates_seq_no_reverse_1_yad = (
        pd.concat([seq_no_reverse_1_yad, cantidates], ignore_index=True)
        .groupby(['session_id', 'yad_no'], as_index=False)['rank'].min()
        .sort_values(['session_id', 'rank'])
        .groupby('session_id').head(10)
        .assign(rank = lambda df: df.groupby('session_id').cumcount() + 1)
    )

    return cantidates_seq_no_reverse_1_yad

In [ ]:
def map10(cantidates, label):
    return (
        cantidates
        .merge(label.assign(label = 1), how='left', on=['session_id', 'yad_no'])
        .assign(label = lambda df: df['label'].mask(df['label'].isna(), 0))
        .assign(precision = lambda df: 1 / df['rank'] * df['label'])
        .groupby('session_id').head(10)
        .groupby('session_id')['precision'].sum().mean()
    )

# 後処理なし
print(map10(valid_cantidates, valid_label))
# 後処理あり
print(map10(add_seq_no_reverse_1_yad(valid_cantidates, valid_log), valid_label))

In [ ]:
test_cantidates = add_seq_no_reverse_1_yad(test_cantidates, test_log)

## 提出

In [ ]:
submission = test_cantidates.pivot(index='session_id', columns='rank', values='yad_no')
submission = submission[[i + 1 for i in range(10)]].copy()
submission = submission.loc[test_session['session_id'], :].copy()
submission.columns = sample_submission.columns

In [ ]:
submission.to_csv(f'submission.csv', index=False, header=True)